In [8]:
np.arange(envWidth)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [7]:
envWidth = 16
envHeight = 8
print(( (np.arange(envWidth) + 0.5) / envWidth - 0.5 ))
print(( (np.arange(envHeight) + 0.5) / envHeight))

[-0.4688 -0.4062 -0.3438 -0.2812 -0.2188 -0.1562 -0.0938 -0.0312  0.0312
  0.0938  0.1562  0.2188  0.2812  0.3438  0.4062  0.4688]
[0.0625 0.1875 0.3125 0.4375 0.5625 0.6875 0.8125 0.9375]


In [5]:
# borrowed from Total3D repo -> notebooks/parse_OR_labels-lighting-reindexed%2Bassigned-V4.ipynb
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train')

import os
import json

%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib widget

# dataset = 'sunrgbd'
dataset = 'OR'

mode = 'val'
# mode = 'train'
# mode = 'train-tmp'

# split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': 'utils_OR/openrooms/preprocessed-full'}
split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': '/home/ruizhu/Documents/Projects/semanticInverse/train/data/openrooms/list_OR_V4full/list'}

if dataset == 'sunrgbd':
    split_file = os.path.join(split_path_dict[dataset], mode + '.json')
    with open(split_file) as file:
        split = json.load(file)
elif dataset == 'OR':
    split_file = os.path.join(split_path_dict[dataset], mode + '.txt')
    with open(split_file) as f:
        mylist = f.read().splitlines() 
    split = [x.split(' ') for x in mylist]

print(len(split), split[0])

5664 ['scene0509_00', '1', 'mainDiffLight_xml1/scene0509_00/im_1.hdr', 'main_xml1/scene0509_00/imsemLabel_1.npy']


In [38]:
import torch
import numpy as np
from pathlib import Path
from utils.utils_total3D.utils_OR_visualize import format_layout, Box
from utils.utils_total3D.data_config import OR4XCLASSES_dict, NYU40CLASSES
from utils.utils_total3D.utils_others import OR4X_mapping_catInt_to_RGB, OR4X_mapping_catStr_to_RGB
from utils.utils_total3D.utils_OR_vis_labels import read_vis_scene_3d, RGB_to_01
import pickle

OR = 'OR45'
classes = OR4XCLASSES_dict[OR] if dataset == 'OR' else NYU40CLASSES
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catInt_to_RGB['light']), 'rb') as f:
    OR_mapping_catInt_to_RGB = pickle.load(f)[OR]
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catStr_to_RGB['light']), 'rb') as f:
    OR_mapping_catStr_to_RGB = pickle.load(f)[OR]

import random
image_id = random.randint(0, len(split))
# image_id = 3
pickle_path_ori = split[image_id]

pickle_id = random.randint(0, len(split))
pickle_id = 13
naming = split[pickle_id][2]
# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'mainDiffLight_xml1/scene0017_02/im_4.hdr'
# naming = 'mainDiffMat_xml1/scene0695_03/im_26.hdr'
# pickle_path_ori = '/data/ruizhu/OR-V3-OR45_total3D_train_test_data/%06d.pkl'%pickle_id
pickle_path_ori = '/data/ruizhu/OR-V4full-detachEmitterRERE-OR45_total3D_train_test_data/%s/%s/%s'%(naming.split('/')[0], naming.split('/')[1], naming.split('/')[2].replace('im', 'layout_obj').replace('hdr', 'pkl'))


grid_size = 8

pickle_path = pickle_path_ori.replace('.pkl', '_reindexed.pkl')
pickle_emitters_path = pickle_path_ori.replace('.pkl', '_emitters.pkl')
pickle_emitters_assign_info_list_path = pickle_path_ori.replace('.pkl', '_emitters_assign_info_%dX%d_V4.pkl'%(grid_size, grid_size))

print(pickle_path)

/data/ruizhu/OR-V4full-detachEmitterRERE-OR45_total3D_train_test_data/mainDiffMat_xml1/scene0218_01/layout_obj_1_reindexed.pkl


In [39]:
import pickle
with open(pickle_path, 'rb') as f:
    sequence = pickle.load(f)
print(sequence.keys())

with open(pickle_emitters_path, 'rb') as f:
    sequence_emitters = pickle.load(f)
print(sequence_emitters.keys(), sequence_emitters['sequence_name'], sequence_emitters['withinsequence_id'])
with open(pickle_emitters_assign_info_list_path, 'rb') as f:
    sequence_emitter2wall_assign_info_dict = pickle.load(f)
print(sequence_emitter2wall_assign_info_dict.keys())

import pickle5 as pickle
emitter_representation_type = '0ambient'
emitters_prop_dict_representation_dict_path = pickle_path.replace('layout_obj', ('emitters_prop_dict_%s'%emitter_representation_type)).replace('_reindexed', '')
with open(emitters_prop_dict_representation_dict_path, 'rb') as f:
    emitters_prop_dict_representation_dict = pickle.load(f)


%matplotlib widget

gt_boxes_list = [format_layout(x) for x in sequence['boxes']['bdb3D']]
for x, class_id in zip(gt_boxes_list, sequence['boxes']['size_cls']):
    x.update({'class_id': class_id})
gt_boxes_dict = {}
for key in list(gt_boxes_list[0].keys()):
    gt_boxes_dict[key] = [x[key] for x in gt_boxes_list]
    
gt_layout = sequence['layout']['bdb3D']

from utils.utils_total3D.data_config import Dataset_Config
from utils.utils_total3D.utils_OR_layout import to_dict_tensor

from utils.utils_total3D.utils_OR_cam import get_rotation_matrix_gt

OR = 'OR45'
dataset_config = Dataset_Config(dataset, OR=OR, version='V3', paths={'total3D_lists_path': '../train/data/openrooms/list_OR_V4full', 'layout_emitter_path': '/data/ruizhu/OR-V4full-OR45_total3D_train_test_data'})
bins_tensor = to_dict_tensor(dataset_config.bins, if_cuda=False)
cam_R_gt = get_rotation_matrix_gt(bins_tensor,
                                  torch.zeros([1]).long()+sequence['camera']['pitch_cls'], torch.zeros([1])+sequence['camera']['pitch_reg'],
                                  torch.zeros([1]).long()+sequence['camera']['roll_cls'], torch.zeros([1])+sequence['camera']['roll_reg']).squeeze().numpy()

# ======== emitters_list
emitters_obj_list = []
for x in range(sequence_emitters['boxes']['bdb3D'].shape[0]):
    obj_dict_new = {'obj_box_3d': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'emitter_prop': sequence_emitters['boxes']['emitter_prop'][x], \
                    'bdb3D_emitter_part': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'cat_id': sequence_emitters['boxes']['size_cls'][x], \
                    'cat_name': classes[sequence_emitters['boxes']['size_cls'][x]], 'cat_color': RGB_to_01(OR_mapping_catInt_to_RGB[sequence_emitters['boxes']['size_cls'][x]])}
    obj_random_id = sequence_emitters['boxes']['random_id'][x]
    emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']
    if sequence_emitters['boxes']['emitter_prop'][x]['obj_type'] == 'window':
        light_center_world_total3d = emitter_prop_total3d['light_center_world_total3d'].reshape(3, 1)
        light_axis_world_total3d = emitter_prop_total3d['light_axis_world_total3d'].reshape(3, 1)
    else:
        light_center_world_total3d = np.zeros((3, 1), dtype=np.float32)
        light_axis_world_total3d = np.zeros((3, 1), dtype=np.float32)
    obj_dict_new['light_world_total3d_centeraxis'] = [light_center_world_total3d, light_axis_world_total3d]
    
    obj_dict_new['emitter_prop'].update({'emitter_rgb_float': emitter_prop_total3d['intensity']})

    emitters_obj_list.append(obj_dict_new)
    
emitter2wall_assign_info_list = sequence_emitter2wall_assign_info_dict['emitter2wall_assign_info_list']
cell_info_grid_GT_includeempty = sequence_emitter2wall_assign_info_dict['cell_info_grid']
cell_info_grid_GT = []
for wall_idx in range(6):
    for i in range(grid_size):
        for j in range(grid_size):
            cell_info = cell_info_grid_GT_includeempty[wall_idx * grid_size**2 + i * grid_size + j]
            if cell_info['obj_type'] is None:
                continue
            cell_info['wallidx_i_j'] = (wall_idx, i, j)
            cell_info_grid_GT.append(cell_info)
                
            obj_random_id = cell_info['emitter_info']['random_id']
            emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']


            cell_intensity = np.array([emitter_prop_total3d['intensity_scale'] * x * 255.for x in emitter_prop_total3d['intensity_scaled']]) # intensity_scaled: [0., 1.]
            cell_info['emitter_info']['intensity_scalelog'] = np.log(np.clip(np.linalg.norm(cell_intensity.flatten()) + 1., 1., np.inf))
            cell_info['emitter_info']['intensity'] = emitter_prop_total3d['intensity']
            # print(cell_intensity[wall_idx, i, j], cell_info['emitter_info']['intensity'])
            cell_info['emitter_info']['intensity_scale'] = emitter_prop_total3d['intensity_scale']
            cell_info['emitter_info']['intensity_scaled'] = emitter_prop_total3d['intensity_scaled']

            # other representation-specific params
            if cell_info['obj_type'] == 'window':
                cell_info['emitter_info']['lamb'] = emitter_prop_total3d['lamb']
                cell_info['emitter_info']['light_dir_abs'] = emitter_prop_total3d['light_axis_world_total3d'].reshape(3,)
                if emitter_representation_type in ['1ambient']:
                    cell_info['emitter_info']['ambient'] = emitter_prop_total3d['ambient']
                if emitter_representation_type in ['2ambient']:
                    cell_info['emitter_info']['ambientL'] = emitter_prop_total3d['ambientL']
                    cell_info['emitter_info']['ambientR'] = emitter_prop_total3d['ambientR']



plt.figure(figsize=(15, 8))
ax_2d = plt.gca()

grid_prob = sequence_emitter2wall_assign_info_dict['wall_grid_prob'].reshape(6, -1)
import imageio
rgb_img = imageio.imread(sequence['rgb_img_path'])

scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, None, gt_layout, None, gt_boxes_dict, None, 'GT', None, dataset=dataset, OR='OR45', \
                emitters_obj_list=emitters_obj_list, emitter2wall_assign_info_list=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=np.random.random((6, grid_size**2)), emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl'})
if_use_plt=True
img_map, _ = scene_box.draw_projected_layout('GT', return_plt=True, if_save=False, save_path='', if_use_plt=if_use_plt, fig_or_ax=ax_2d)
img_map2 = scene_box.draw_projected_bdb3d('GT', return_plt=True, if_save=False, save_path='', if_use_plt=if_use_plt, fig_or_ax=ax_2d)
# plt.show()

fig_3d, ax_3ds_scene = scene_box.draw_3D_scene_plt('GT', if_show_objs=False)
# ax_3d = ax_3ds[0]
# ax_3d[0].view_init(elev=-46, azim=86)

# %matplotlib widget
# sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train/SimpleLayout')
# depth_combined, mask_conflict = scene_box.draw_projected_depth('GT', return_plt=True, if_save=False, save_path='', if_vis=True)


dict_keys(['rgb_img_path', 'depth_map', 'boxes', 'camera', 'layout', 'scene_name', 'withinsequence_id', 'meta_split', 'meta_name', 'sub_name', 'scene_pickle_file', 'cam_pickle_file', 'frame_pickle_file', 'reindex_info_dict'])
dict_keys(['sequence_name', 'withinsequence_id', 'boxes', 'emitter_random_ids']) scene0218_01 1
dict_keys(['emitter2wall_assign_info_list', 'emitters_obj_list', 'wall_grid_prob', 'cell_prob_mean', 'cell_prob', 'cell_count', 'cell_info_grid'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[draw_projected_layout] Returned.
[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …